In [2]:
import numpy as np
import glob
import json
from tqdm.notebook import tqdm

from skimage import color

from utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate, SEMDataset

from crdp import rdp
from skimage import io, filters, morphology, util
from torch.utils.data import Dataset

from matplotlib import pyplot as plt
import os
from pathlib import Path
import json

## Углы

In [ ]:

images = np.load('images_7_classes_mixer_preprocess.npy')
names = np.load('metadata_7_classes_mixer_preprocess.npy')
names = [name.split('\\')[-1] for name in names]

folder = 'data/'

types_dict = {'2550-51': '2550 призм, кадр 51\n доля карбина 54.51%',
              '2550-52': '2550 призм, кадр 52\n доля карбина 50.75%',
              '2550-53': '2550 призм, кадр 53\n доля карбина 45.81%',
              '3400-51': '3400 призм, кадр 51\n доля карбина 53.53%',
              '3400-52': '3400 призм, кадр 52\n доля карбина 50.47%',
              '3400-53': '3400 призм, кадр 53\n доля карбина 45.88%',
              '3400-54': '3400 призм, кадр 54\n доля карбина 40.88%'}

dens_dict={'2550-51': 0.5451066158234127,
           '2550-52': 0.5075556821469908,
           '2550-53': 0.4580166661499669,
           '3400-51': 0.535331783234127,
           '3400-52': 0.504782962859623,
           '3400-53': 0.45885131190062833,
           '3400-54': 0.40882115988756607}



In [ ]:
def angles_approx_save(images_path, save_path, types_dict, step, max_images_num_per_class=None, no_cache=False):
    """
    :param save_path:
    :param images: ndarray uint8 [[image1_class1,image2_class1,..],[image1_class2,image2_class2,..]..]
    :param paths:
    :param types_dict: list str [class_type1,class_type2,..]
    :param step: scalar int [0,N]
    :param max_images_num_per_class:
    """
    #
    # вычисление и сохранение распределения углов для всех фотографий одного образца
    #

    json_data = []
    
    dataset = SEMDataset(images_path, no_cache=no_cache, max_images_num_per_class=max_images_num_per_class)
        

    for i in tqdm(range(dataset.images_paths.shape[0])):
        all_angles = []
        # all_unique_angels=dict()
        
        for j in tqdm(range(dataset.images_paths.shape[1])):
            image, path = dataset.__getitem__(i,j)
            ang=grainMark.get_angles(image)
            
            # all_unique_angels[j]=ang
            all_angles.extend(ang)

        x, y = grainStats.stats_preprocess(all_angles, step)

        (x_gauss, y_gauss), mus, sigmas, amps = grainApprox.bimodal_gauss_approx(x, y)
        name = path.split('/')[-2]

        text = grainGenerate.angles_legend(dataset.images_paths.shape[1], types_dict[name], types_dict[name], step, mus, sigmas,amps, len(all_angles) )
        
        path='/'.join(path.split('/')[:-1])

        json_data.append({'path': path,
                          'name': name,
                          'type': types_dict[name],
                          'legend': text,
                          'density_curve_scatter': [x,y],
                          'gauss_approx_plot': [x_gauss, y_gauss],
                          'gauss_approx_data': {'mus': mus, 'sigmas':sigmas, 'amps':amps},
                          # 'angles_series': all_unique_angels,
                          })

    with open(f'{save_path}_step_{step}_angles.json', 'w', encoding='utf-8') as outfile:
        json.dump(json_data, outfile, cls=grainGenerate.NumpyEncoder, ensure_ascii=False)

In [ ]:
%%time

images_path = '../ml/diffusion/data/o_bc_left_4x_768_360_768_median_generated'
# images_path = '../ml/diffusion/data/o_bc_left_4x_768_360_512_median_generated'
# images_path = '../ml/diffusion/data/o_bc_left_4x_768_360_256_median_generated'

# name = 'o_bc_left_4x_768_360_median_generated_preprocess'
# name = 'o_bc_left_4x_768_360_256_median_generated_preprocess'
# file_name='data/angles/o_bc_left_4x_768_360_median_generated_preprocess'
# file_name='o_bc_left_4x_768_360_256_median_generated_preprocess'

name = 'o_bc_left_4x_768_360_768_median_generated'


types_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}

for step in tqdm(range(5, 6)):
    angles_approx_save(
                        images_path=images_path,
                        save_path=name,
                        types_dict=types_dict,
                        step=step,
                        max_images_num_per_class=None, 
                        no_cache=False
                    )

## Полуоси

In [31]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.float64) or isinstance(obj, np.float32):
            return float(obj)
        if isinstance(obj, np.int64) or isinstance(obj, np.int32): 
            return int(obj)  
        if isinstance(obj, np.ndarray): 
            return list(obj)  
        return json.JSONEncoder.default(self, obj)

def diametr_approx_save(images_path, save_path, types_dict, step, pixel, start=2, end=-3,
                        debug=False, max_images_num_per_class=None, no_cache=False):
    #
    # вычисление и сохранение распределения длин а- и б- полуосей и угла поворота эллипса для разных образцов
    #

    json_data = []
    
    dataset = SEMDataset(images_path, no_cache=no_cache, max_images_num_per_class=max_images_num_per_class)

    for i in tqdm(range(dataset.images_paths.shape[0])):

        all_a_beams = []
        all_b_beams = []
        # all_angles = []
        # all_contours = []

        for j in tqdm(range(dataset.images_paths.shape[1])):
            image, path = dataset.__getitem__(i,j)
            b_beams, a_beams, angles, cetroids, contours = grainMark.get_mvee_params(image, 0.2, debug=debug)

            all_a_beams.extend(a_beams)
            all_b_beams.extend(b_beams)
            # all_angles.extend(angles)
            # all_contours.extend((contours))

        x1, y1 = grainStats.stats_preprocess(all_a_beams, step)
        x2, y2 = grainStats.stats_preprocess(all_b_beams, step)

        angles_set, angles_dens_curve = grainStats.stats_preprocess(np.rad2deg(angles).astype('int32'), step=step)

        x1 = x1.reshape(-1, 1) * pixel
        x2 = x2.reshape(-1, 1) * pixel
        # x_angles = np.array([angles_set]).reshape(-1, 1)

        y1 = np.log(y1).reshape(-1, 1)
        y2 = np.log(y2).reshape(-1, 1)
        # y_angles = angles_dens_curve.reshape(-1, 1)

        x1 = x1[start:end]
        x2 = x2[start:end]
        # x_angles = x_angles[start:end]

        y1 = y1[start:end]
        y2 = y2[start:end]
        # y_angles = y_angles[start:end]

        (x_pred1, y_pred1), k1, b1, angle1, score1 = grainApprox.lin_regr_approx(x1, y1)
        (x_pred2, y_pred2), k2, b2, angle2, score2 = grainApprox.lin_regr_approx(x2, y2)

        dist_step = pixel * step

        name = path.split('/')[-2]

        # dist1_set_mean = np.sum(dist1_set*dens1_curve) * pixel
        # dist2_set_mean = np.sum(dist2_set*dens2_curve) * pixel
        
        dist1_set_mean = 0
        dist2_set_mean = 0

        legend1 = grainGenerate.beams_legend(0, name, types_dict[name], len(all_a_beams), k1, angle1, b1, score1, dist_step, dist1_set_mean)
        legend2 = grainGenerate.beams_legend(0, name, types_dict[name], len(all_b_beams), k2, angle2, b2, score2, dist_step, dist2_set_mean)
        
        path='/'.join(path.split('/')[:-1])

        json_data.append({'path': path,
                          'name': name,
                          'type': types_dict[name],
                          'legend': [{'a_beams': legend1, 'b_beams': legend2}],
                          'density_curve_scatter': [
                              {'a_beams': (x1.flatten(), y1.flatten()),
                               'b_beams': (x2.flatten(), y2.flatten()),
                               # 'angles': (x_angles.flatten(), y_angles.flatten())
                               }
                          ],
                          'linear_approx_plot': [{'a_beams': (x_pred1.flatten(), y_pred1.flatten()),
                                                  'b_beams': (x_pred2.flatten(), y_pred2.flatten())}],
                          'linear_approx_data': [{'a_beams': {'k': k1, 'b': b1, 'angle': angle1, 'score': score1},
                                                  'b_beams': {'k': k2, 'b': b2, 'angle': angle2, 'score': score2}}],
                          # 'beams_length_series': [{'a_beams': all_a_beams, 'b_beams': all_b_beams}],
                          # 'angles_series': all_angles,
                          # 'contours_series': all_contours,
                          'pixel2meter': pixel,
                          })

    with open(f'{save_path}_step_{step}_beams.json', 'w', encoding='utf-8') as outfile:
        json.dump(json_data, outfile, cls=NumpyEncoder, ensure_ascii=False)


In [32]:
pixel = 50 / 1000

images_path = '../ml/diffusion/data/o_bc_left_4x_768_360_768_median_generated'
# images_path = '../ml/diffusion/data/o_bc_left_4x_768_360_512_median_generated'
# images_path = '../ml/diffusion/data/o_bc_left_4x_768_360_256_median_generated'

# name = 'o_bc_left_4x_768_360_median_generated_preprocess'
# name = 'o_bc_left_4x_768_360_256_median_generated_preprocess'
# file_name='data/angles/o_bc_left_4x_768_360_median_generated_preprocess'
# file_name='o_bc_left_4x_768_360_256_median_generated_preprocess'

# name = 'o_bc_left_4x_768_360_768_median_generated'
save_path = 'test'


types_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}


for step in range(5, 6):
    # grainGenerate.diametr_approx_save(
    diametr_approx_save(
                        images_path=images_path,
                        save_path=save_path,
                        types_dict=types_dict,
                        step=step,
                        max_images_num_per_class=100, 
                        no_cache=False,
                        pixel = pixel
    )
    # diametr_approx_save(save_path=name, images=images,paths=paths, types_dict=types_dict, step=step, pixel=pixel,max_images_num_per_class=100)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]